# IMU Data

Begining with acceleration, we can look at the data from our primary IMU.


## Acceleration

We start off looking at the vertical component (in body frame) of the measured acceleration during the flight.

In [ ]:
from numpy import array, subtract, divide, multiply, median, std, var
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import launch

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"IMU Vertical Acceleration (Measured)")
plt.ylabel(r"Measured Acceleration [m/s${}^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(launch.adis.time[200:-200], launch.adis.acc_x_filter[200:-200], alpha=0.75)

plt.xlim([-5,42])
#ax1.legend(loc=1)
plt.show()

We 'see' gravity (~9.8 m/s$^2$) at the begining, followed by the main thrust of the rocket motor. Then the acceleration goes negative (due to drag after the motor burns out) and stays around 0 (in freefall) until the chute's open and there is chaos. This is a pretty classic acceleration curve from a high powered rocket.


In [ ]:
apeak = max(launch.adis.acc_x_filter[200:-200])
abreak = min(launch.adis.acc_x_filter[200:-200])
print """During boost we see a peak accleration of %0.0f m/s/s (%0.1f g)
and right after burnout the aerodynamic drag on the rocket immediatly
pulls us back to %0.1f m/s/s (%0.1f g).""" % (apeak, apeak/launch.g_0, abreak, abreak/launch.g_0)

## Velocity

We can take this vertical acclereation data (subtract gravity!) and integrate it we get vertical veloctcity.

In [ ]:
from scipy.integrate import simps

tacc_x = subtract(launch.adis.acc_x, launch.g_0)

velocity = [0, 0]
for i in range(2, len(launch.adis.acc_x)):
    velocity.append(velocity[-1] + simps(tacc_x[i-2:i], launch.adis.time[i-2:i]))

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"IMU Integrated Vertical Velocity")
plt.ylabel(r"Integrated Velocity [m/s]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(launch.adis.time, velocity, alpha=0.75, label="velocity")

ax2 = ax1.twinx()
ax2.set_ylabel(r"Mach")
ax2.plot(launch.adis.time, divide(velocity, launch.M), alpha=0)
ax2.plot([-50, 100], [1.0, 1.0], 'k-.', alpha=0.3, label="Speed of Sound")
ax2.grid(b=False)
#ax2.set_ylim([-50,150])

plt.xlim([-5,42])
#ax1.legend(loc=1)
ax2.legend(loc=1)
plt.show()

In [ ]:
vpeak = max(velocity)
print """This shows a peak velocity of %0.0f m/s (Mach %0.1f). That's a max velocity
of just over the speed of sound!
""" % (vpeak, vpeak/launch.M)

## Altitude

And now that we have velocity we integrate a second time to solve for altitude. Note that the rocket started at about 1,390 meters above sealevel, this is altitude above ground (AGL), not sea level reference.

In [ ]:
alt = [0, 0]
for i in range(2, len(launch.adis.acc_x)):
    alt.append(alt[-1] + simps(velocity[i-2:i], launch.adis.time[i-2:i]))

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"IMU Integrated Altitude")
plt.ylabel(r"Integrated Altitude AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(launch.adis.time, divide(alt,1000.0), alpha=0.75, label="Altitude")

plt.xlim([-5,42])
#ax1.legend(loc=1)
plt.show()

In [ ]:
altpeak = max(alt)
print """Peak altitude of the rocket meausured with this method is %0.2f km AGL (%0.2f km MSL).
""" % (altpeak/1000.0, (altpeak+launch.rail_alt)/1000.0)

It's important to note that the integrated results above (velocity and acceleration) are preliminary and don't inlcude full solutions or filting and data fusion. Interation error will creep in quickly. A full solution is presented later in this paper.

In [ ]:
# Write intermediate results for future use
with open('uncalibrated_integrated_velocity.csv', 'w') as fout:
    for i, t in enumerate(launch.adis.time):
        fout.write(",".join(["%0.12f"%t, "%0.3f"%velocity[i]]))
        fout.write("\n")

with open('uncalibrated_integrated_altitude.csv', 'w') as fout:
    for i, t in enumerate(launch.adis.time):
        fout.write(",".join(["%0.12f"%t, "%0.3f"%alt[i]]))
        fout.write("\n")